In [1]:
import pandas as pd
from checkStationary import ADF
import numpy as np
from dateutil import relativedelta
from config import getClient

In [2]:
datalib = getClient("MC")

In [3]:
def getData(typ):
    assert (typ in ["P", "R"])
    if typ == "P":
        df = pd.read_csv(datalib+"Processed.csv")
    else:
        df = pd.read_csv(datalib+"Reported.csv")
    
    df["Date"] = pd.to_datetime(df['Month'])
    del df["Month"]
    
    cols = ["Country", "Program", "Parent ICA", "Driver", "Date", "Value"]
    df = df[cols]
    
    sort = ["Country", "Program", "Parent ICA", "Driver", "Date"]
    df = df.sort_values(sort)
    df = df.set_index(["Country", "Program", "Parent ICA", "Driver"], drop=True)
    return df

In [4]:
def writeResults(pkid, typ):
    DELIM = "|"
    assert (typ in ["P", "R"])
    if typ == "P":
        filenm = "Processed+Results.csv"
    else:
        filenm = "Reported+Results.csv"
    with open(datalib+filenm, "w") as output:
        hdr = "Country" +DELIM+ "Program"+DELIM+"Parent ICA"+DELIM+"Driver"+\
        DELIM+"Length"+DELIM+"Density"+DELIM+"Stationary"
        hdr += "\n"
        output.write(hdr)
        for key, val in pkid.items():
            key = key[0] +DELIM+ key[1] +DELIM+ key[2] +DELIM+ key[3]
            rec = str(val["length"]) +DELIM+ str(val["density"]) +DELIM+ str(val["stationary"])
            rec = key +DELIM+ rec + "\n"
            output.write(rec)

In [5]:
def processDF(df, typ):
    pkid    = {}

    prt = False
    CI = "5%"

    for idx, val in df.groupby(level=[0, 1, 2, 3]):
        results = {}
        amounts = val["Value"]
        results["length"] = len(amounts)
        amounts = amounts.reset_index(drop=True)
        if results["length"] > 8:
            results["stationary"] = ADF(amounts, CI, prt)
        else:
            results["stationary"] = None
        dates = val["Date"]
        dates = dates.reset_index(drop=True)
        diff = relativedelta.relativedelta(dates.iloc[-1], dates[0])
        duration = 12*diff.years + diff.months
        if duration > 0:
            if typ == "R":
                results["density"] = (len(dates)-1)*3  / duration
            else:
                results["density"] = (len(dates)-1)  / duration
        else:
            results["density"] = None
        pkid[idx] = results
    return pkid

In [7]:
typ = "R"
df = getData(typ)
results = processDF(df, typ)
writeResults(results, typ)

typ = "P"
df = getData(typ)
results = processDF(df, typ)
writeResults(results, typ)

/home/tbrownex/tensorflow/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:1510: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/home/tbrownex/tensorflow/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:846: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/tbrownex/tensorflow/lib/python3.5/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
